# DSPy in Financial Services
# Risk assessment with DSPy

DSPy makes it easy to create, evaluate, and deploy GenAI applications in a repeatable and scalable way.

In Financial Services, like in other regulated industries, it is crucial to ensure that outcomes produced by AI applications comply with laws, regulations, and ethical guidelines. In particular there are 3 aspects to consider:

1. **Explainability**: It is crucial to understand why an AI system made a particular decision or prediction. Outcomes must also be fair and non-discriminatory.
2. **Reliability**: It is important that regardless of the complexities of the processes that lead to an outcome, the elements along the way reliably function as intended.
3. **Consistency**: It is of paramount importance that outcome quality is consistent and reliable across different invocations of the system and there is no room for hallucinations.

In this notebook we will see how to use DSPy to create a risk assessment agent.

We will use the following optimizers to progress through the use case and demonstrate their differences:

- **Zero Shot**: This is the simplest form of prompting. We only provide with a description of the task (instruction).
- **Labeled Few Shot**: This is a more advanced form of prompting. We provide a description of the task (instruction), and a few labelled examples for the model to learn from and follow.
- **Bootstrap Few Shot**: This optimizer goes a level up in terms of performance by introducing the use of metrics to generate complete demonstrations of the program and optimize against them.
- **MIPRO v2**: Finally, this optimizer leverages Bayesian Optimization to optimize both instructions and few shot examples simultaneously by finding the best combination of instructions and examples.

## Zero Shot

In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy import ChainOfThought
from dsp import Claude
from typing import List, Optional
import pandas as pd
import textwrap

In [2]:
import os
os.environ["DSP_CACHEBOOL"] = "False"
import phoenix as px
from phoenix.trace import using_project

phoenix_session = px.launch_app()
from openinference.instrumentation.dspy import DSPyInstrumentor
from openinference.instrumentation.anthropic import AnthropicInstrumentor

from phoenix.otel import register
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = register(endpoint=endpoint, set_global_tracer_provider=False)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

DSPyInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [3]:
# Initialize the language model
#worker = dspy.OpenAI(model="gpt-3.5-turbo", model_type="chat", max_tokens=3000)
worker = Claude(model="claude-3-5-sonnet-20240620", max_tokens=3000)
#worker = dspy.Cohere(model="command-r-plus", max_tokens=3000)
#worker = dspy.HFModel(model = 'mistralai/Mistral-7B-Instruct-v0.2', max_tokens=3000)
dspy.configure(lm=worker)
dspy.settings.configure(backoff_time = 60)

In [4]:
# Input data
applicant_info = """
Name: John Doe
Age: 35
Annual Income: $75,000
Credit Score: 720
Existing Debts: $20,000 in student loans, $5,000 in credit card debt
Loan Amount Requested: $250,000 for a home mortgage
Employment: Software Engineer at Tech Corp for 5 years
"""

In [5]:
class ZeroShot(dspy.Module):
    """
    You are given a piece of text that contains information about an applicant. 
    Analyze the applicant's financial information and return a risk assessment.
    """
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, applicant):
        return self.prog(question="Analyze the applicant's financial information and return a risk assessment. Applicant: " + applicant)

In [6]:
with using_project("zero_shot"):
    module = ZeroShot()
    response = module(applicant_info)
    print(f"ZeroShot response:\n {response}")

    prompt_used = worker.inspect_history(n=1)
    print(f"Prompt used: {prompt_used}")

 		You are using the client Claude, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


ZeroShot response:
 Prediction(
    answer="Based on the provided financial information for John Doe, here's a risk assessment:\n\nAnswer: Low to Moderate Risk\n\nReasons for this assessment:\n\n1. Positive factors:\n   - Stable employment: 5 years at Tech Corp as a Software Engineer\n   - Good credit score: 720 is considered good and indicates responsible credit management\n   - Solid annual income: $75,000 is a respectable salary for a 35-year-old professional\n   - Debt-to-income ratio: Existing debts are relatively low compared to annual income\n\n2. Potential concerns:\n   - Loan amount requested ($250,000) is more than 3 times the annual income, which is on the higher side\n   - Existing debts, while not excessive, do add to the overall financial obligations\n\n3. Recommendations:\n   - Verify the applicant's ability to make consistent mortgage payments\n   - Consider the local housing market and ensure the loan amount is appropriate for the property value\n   - Review the applic

## Labeled Few Shot

In [7]:
import json
from dspy.teleprompt import LabeledFewShot

In [8]:
class RiskAssessment(dspy.Signature):
    """Analyze the applicant's financial information and return a risk assessment."""
    question = dspy.InputField()
    applicant = dspy.InputField()
    answer = dspy.OutputField(desc="""
                              A thorough risk analysis about the applicant, justifying the assessment 
                              for each of the parameters considered from the applicant
                              """
                              )

In [9]:
class RiskAssessmentAgent(dspy.Module):
    def __init__(self):
        self.question = "Analyze the applicant's financial information and return a risk assessment."
        self.assess_risk = ChainOfThought(RiskAssessment, n=3)
    def forward(self, applicant:str):
        question = self.question
        applicant = applicant
        pred = self.assess_risk(question=question, applicant=applicant)

        return dspy.Prediction(answer = pred.answer)

In [10]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [11]:
# Train
teleprompter = LabeledFewShot()
lfs_optimized_advisor = teleprompter.compile(RiskAssessmentAgent(), 
                                             trainset=trainset[3:]
                                            )

In [12]:
with using_project("labeled_few_shot"):
    response = lfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"LabeledFewShot Optimised response:\n {wrapped_response}")

LabeledFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Low to Medium Risk.   Justification: 1. John's good
credit score (720) indicates a history of responsible credit
management. 2. His stable employment as a Software Engineer for 5
years suggests job security and income stability. 3. The annual income
of $75,000 provides a solid foundation for loan repayment. 4. The
existing debts are manageable, though the addition of a mortgage will
significantly increase his debt load. 5. The loan-to-income ratio for
the requested mortgage is within typical lending standards.  However,
there are some factors to consider: 1. The potential debt-to-income
ratio after the mortgage (52.42%) is on the higher side, which could
strain his finances. 2. The $5,000 in credit card debt, while not
excessive, should be monitored.  Overall, John Doe appears to be a
responsible borrower with a stable financial situation. The main
concern is the potent

In [13]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment
Applicant: Name: Richard Turner Age: 45 Annual Income: $95,000 Credit Score: 620 Existing Debts: $20,000 in credit card debt Loan Amount Requested: $30,000 for home renovations Employment: Engineer at a manufacturing company for 15 years
Answer: Based on the analysis of Richard Turner's financial information, the risk assessment is Medium. Here's the justification for this assessment: 1. Income: Richard's annual income of $95,000 is relatively high, providing a good foundation for loan repayment. 2. Employment Stability: 15 

## Bootstrap Few Shot

In [14]:
from dspy.teleprompt import BootstrapFewShot

In [15]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a risk assessment along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

In [16]:
def risk_assessment_metric(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer

    correct = f"The text above should provide a risk assessment for `{applicant}`. Does it do so? Answer with Yes or No."
    complete = f"Does the text above make for a reasoned assessment across all areas mentioned in `{applicant}`? Answer with Yes or No."

    with dspy.context(lm=worker):
        correct =  dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=correct)
        complete = dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=complete)

    correct, complete = [m.assessment_answer.split()[0].lower() == 'yes' for m in [correct, complete]]
    score = (correct + complete) if correct else 0

    if trace is not None: return score >= 2
    return score / 2.0

In [17]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [18]:
with using_project("bootstrap_few_shot"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized = BootstrapFewShot(metric=risk_assessment_metric, **config)
    bfs_optimized_advisor = bfs_optimized.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

 56%|█████▌    | 15/27 [10:07<08:05, 40.47s/it]

Bootstrapped 4 full traces after 16 examples in round 0.


In [19]:
with using_project("bootstrap_few_shot"):
    response = bfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"BootstrapFewShot Optimised response:\n {wrapped_response}")

BootstrapFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium Risk. Here's the justification for this
assessment:  1. Income and Employment: John's stable job as a Software
Engineer with 5 years of experience and a solid annual income of
$75,000 provide a good foundation for loan repayment.  2. Credit
Score: His credit score of 720 is good, indicating responsible credit
management and reducing the risk of default.  3. Existing Debts: While
the student loan debt is significant, it's not unusual for his
profession. The credit card debt, however, suggests some potential for
financial strain.  4. Debt-to-Income Ratio: The current DTI of 33.33%
is manageable, but the potential DTI after the mortgage (51.30%) is
concerning. Most lenders prefer a DTI below 43% for qualified
mortgages.  5. Housing Expense Ratio: At 17.97%, this is within the
generally acceptable range of 28% or less.  6. Loan Amount: The
$250,000 mortgage is subs

In [20]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan Employment: Software Engineer at XYZ Corp for 5 years

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce a comprehensive risk assessment for John Smith: 1. Age: At 35, John is in his prime working years, which is a positive factor. 2. Annual Income: $80,000 is a solid income, indicating good earning potential and ability to repay.

In [21]:
bfs_optimized_advisor.save('bfs_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

## Bootstrap Few Shot with Advanced Metrics using G-Eval (https://arxiv.org/abs/2303.16634)

In [23]:
from deepeval.metrics import GEval, BiasMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)


In [39]:
def risk_assessment_metric_adv(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer

    test_case = LLMTestCase(
            input=applicant,
            actual_output=risk_assessment
    )
    bias_metric = BiasMetric(
        threshold=0.7,
        include_reason=True,
        async_mode=False
    )
    bias_metric.measure(test_case)
    answer_relevancy_metric = AnswerRelevancyMetric(
        threshold=0.7,
        include_reason=True,
        async_mode=False
    )
    answer_relevancy_metric.measure(test_case)
    coherence_metric = GEval(
        name="coherence",
        criteria="Coherence - determine if the actual output is coherent with the input",
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        evaluation_steps=[
            "Check if the actual output is logically consistent with the input on the basis of the instructions from the input",
            "Assess if the actual output is relevant to the input and maintains a logical flow on the basis of the instructions from the input",
        ],
        async_mode=False
    )
    coherence_metric.measure(test_case)
    
    
    score = (1 - bias_metric.score) + answer_relevancy_metric.score + coherence_metric.score
    
    if trace is not None: return score >= 3
    return score / 3.0

In [25]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [26]:
with using_project("bootstrap_few_shot_adv_metrics"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized_adv_metrics = BootstrapFewShot(metric=risk_assessment_metric_adv, **config)
    bfs_optimized_advisor_adv_metrics = bfs_optimized_adv_metrics.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

  0%|          | 0/27 [00:00<?, ?it/s]

Output()

Output()

Output()

  4%|▎         | 1/27 [01:04<28:03, 64.73s/it]

...

Output()

Output()

Output()

 15%|█▍        | 4/27 [03:59<22:56, 59.83s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


In [27]:
with using_project("bootstrap_few_shot_adv_metrics"):
    response = bfs_optimized_advisor_adv_metrics(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"BootstrapFewShot Optimised with Advanced Metricsresponse:\n {wrapped_response}")

BootstrapFewShot Optimised with Advanced Metricsresponse:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium Risk. Here's the justification for this
assessment:  1. Income and Employment: John's income is solid, and his
5-year tenure as a Software Engineer demonstrates stability and
potential for future income growth. This is a positive factor in the
risk assessment.  2. Credit Score: His credit score of 720 is good to
very good, indicating responsible credit management and lowering the
risk of default. This is another positive factor.  3. Existing Debts:
While John has existing debts, they are not excessive given his income
and education level. The student loans suggest he has invested in his
career, which is generally positive.  4. Loan Amount: The $250,000
mortgage request is about 3.33 times his annual income, which is
within typical guidelines but on the higher end. This increases the
risk slightly.  5. Debt-to-Income Ratio: The potential new

In [28]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: Jane Doe Age: 28 Annual Income: $65,000 Credit Score: 680 Existing Debts: $15,000 in student loans Loan Amount Requested: $200,000 for a home mortgage Employment: Marketing Manager at ABC Corp for 3 years

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce a thorough risk assessment for Jane Doe: 1. Age: At 28, Jane is relatively young, which could be both positive (potential for income growth) and negative (less financial experience). 2. Annual Income: 

In [29]:
bfs_optimized_advisor_adv_metrics.save('bfs_optimized_advisor_with_adv_metrics_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

## MIPRO

In [30]:
from dspy.teleprompt import MIPROv2

In [47]:
with using_project("mipro_v2_adv_metrics"):
    worker_gpt4o = dspy.OpenAI(model="gpt-4o", model_type="chat", max_tokens=5000)
    dspy.configure(lm=worker_gpt4o)
    mipro_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(num_candidates=10, init_temperature=0.1)
    mipro_optimized = MIPROv2(metric=risk_assessment_metric_adv, **config)
    mipro_optimized_advisor = mipro_optimized.compile(RiskAssessmentAgent(),
                                                trainset=mipro_trainset,
                                                max_bootstrapped_demos=2,
                                                max_labeled_demos=2,
                                                num_trials=5,
                                                minibatch=False
                                                )
    mipro_optimized_advisor.save(f'mipro_optimized_advisor_compiled.json')
    response = mipro_optimized_advisor(applicant_info)
    print(f"MIPRO Optimised response:\n {response}")
    

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 21 examples in val set * 5 batches * # of LM calls in your program = (30 * # of LM calls in your program) task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the proje

  0%|          | 0/6 [00:00<?, ?it/s]

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [48]:
wrapped_response = textwrap.fill(response.answer, width=70)
print(f"MIPRO Optimised response:\n {wrapped_response}")

MIPRO Optimised response:
 Based on the analysis, John Doe presents a Moderate risk for the
$250,000 home mortgage. While he has a good credit score and stable
employment in a promising field, the size of the loan relative to his
income and existing debts means his debt-to-income ratio will
significantly increase. His current financial obligations are
manageable, but the additional mortgage could stretch his financial
capacity. While he appears to be a responsible borrower, the lender
may want to ensure he has a solid plan for managing the increased debt
load. Overall, his risk level is moderate, with a need for careful
monitoring of his capacity to manage the higher debt.


In [49]:
prompt_used = worker_gpt4o.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the provided financial information of the applicant to generate a comprehensive risk assessment. Consider key metrics such as credit score, debts, income, and debt-to-income ratio, as well as qualitative factors like employment stability and potential financial changes. Provide a detailed rationale that explains the step-by-step reasoning process used to evaluate each parameter, and conclude with a justified risk rating ranging from Low to High Risk.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan

In [45]:
mipro_optimized_advisor.save(f'mipro_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 